SIMD: Single instruction multiple data 


Kernel level optimization 
* Operator simplification 
- shift the sqrt to constant value rather than variable (val = sqrt(var); val > const → var>const^2)
- Horner's rule: pull out the commmon factor from polynomial expression
* Calculations Reuse Across Iterations (eg. convolution with stride 1, the calculations from [2,n-1] can eb reused)
* Calculation reuse across iterations
* avoid control code, write code in linear fashion. Not nested for loops
* Fixed point, Q point, so the bits are assigned as  8 integers and 7 fractions
* lookup tables instead of sin, sqrt etc. Quantize float and use LUT 
* Array of Structures (AOS) vs Structure of Arrays (SOA)

Loop Transformations 
* Collapsing smaller inner looop, loop fusion, fission
* Load balancing 


Ref: 

https://www.ti.com/lit/an/spna165/spna165.pdf?ts=1626326802278&ref_url=https%253A%252F%252Fwww.google.com%252F

Dynamic range is the ratio between the largest and smallest values that a certain quantity can assume

Terms:
* Latency, bandwidth, throughput, dynamix range,
* Different scaling for weights, bias (knowledge of dynamic range of alll layers from training)
* per channel scaling: helps provide flexibility in quantization
* Scale only quantization (centered quantization, no offset ) unscaled: you can use unsigned bit and save one bit that doesnt give any imkprovement
* FP16 quantization, int8, mixed precision (in yolo v1, deeper layers should be higher precision as more susceptible to noise )
* Match the output distribution bertween fp32, int8 checking the KL divergence



* Train for quantization (forward pass clip weight to nearest quantized value. Backprop dont clip. Activations unquantized )


Ref: 
https://www.youtube.com/watch?v=VsGX9kFXjbs&t=2159s

![](helper/8.JPG)

Nvidia Yolov1 tensroRT example 

Best from Int8: 
* setDynamicRange 
* mixed precision:
**type constraint

### Covers all quantization 
https://arxiv.org/pdf/2103.13630.pdf

### Quantization aware training 

* Train for quantization (forward pass clip weight to nearest quantized value. Backprop dont clip. Activations unquantized )

![](helper/4.JPG)

Ref : https://www.youtube.com/watch?v=-jBmqY_aFwE 

* Since the data to DNN is kind of noisy and quantization also adds noise, it doestn affect the accuracy number that much. But if the quantization introduces a bias, shifts the distribution it can be detrimental. Until it adds uniform noise thats fine. 
* Since we have a lot of sparsity it is very important to encode 0.0 correctly.

* Quantized inference calculations: We have converted weights into 8 bits, we also convert input to 8 bit (imagine a step functiohn, we floor/ceil to nearest step)
* DSP dont support floating points, but with int8 they can be used

* Symmetric vs asymmetric: When we quantized weights, we were just converting them into an 8 bit and converting them back to FP later. But here we need to do arithematic operations, thus we need to ensure that: 

** zero is always zero
** encoded (-x) should also flip the sign 

So we stored it in signed format with 0 exactly at 0. But this causes the following problems: 
* We have one less bit for the negative range (-127,0) than positive (0,128) which can cause problems
Also, earlier we coulod convert -4.5 to 5.4 to 255 bits. Now even though min value is -4.5 we need to have a range of (-5.4 to 5.4) (as 0 has to be in the middle)


How to find this range (a.k.a dynamic range??): 
* Analyze largest possible range (check max input and min input and calculate what dot product min max will be )
* Training logging
* Post training calibration


We have two 8 bit beign multiply which gives 16 bits and then dot product adds many of them, so we have to kee pthe output of the dot product as 32 bit, but again not the whole range may be used. We can use 16 bit accumulaters to keep track of it (no very clear) 

* Per channel scaling
output_n = input_n*weight_n 
So now weight_n is scaled differently for each channel and we scale  back the output accordingly.

Ref: https://www.youtube.com/watch?v=AgezOkBTV90

![](helper/9.JPG)
![](helper/10.JPG)
![](helper/11.JPG)
Then do huffman encoding



Knowledge Distillation: Student teacher network

* Lottery Ticket Hypothesis: Often a small subset of NN can perform equivalent. So how to fidn that? Randomly prune p% of the smallest weights. If it works thats your network

### M E 

* software pipelining:linear assemblers 
* Redundancy in code: reuse loaded data
* Code partitioning: cluster based DSP 